In [1]:
%load_ext lab_black

In [2]:
%cd ..

/home/shim/cev/dl/motion-keypoint


In [3]:
import argparse
import json
import math
import sys

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from tqdm import tqdm

import networks
import options
import utils
from datasets import KeypointDataset, TestKeypointDataset, get_pose_datasets
from error_list import error_list
from losses import JointMSELoss, KeypointLoss, KeypointRMSE

---

## 하이퍼 파라미터

In [4]:
config = options.load_config("config/hrnet-w48-inference.yaml")

[2021-03-23 10:17:33  INFO] 학습 시작
[2021-03-23 10:17:33  INFO] pose_model: HRNet-W48
[2021-03-23 10:17:33  INFO] result_dir: results/hrnet-train
[2021-03-23 10:17:33  INFO] data_dir: data/ori
[2021-03-23 10:17:33  INFO] lr: 0.0001
[2021-03-23 10:17:33  INFO] batch_size: 10
[2021-03-23 10:17:33  INFO] start_epoch: 1
[2021-03-23 10:17:33  INFO] SAM: False
[2021-03-23 10:17:33  INFO] folds: [1, 2, 3, 4, 5]
[2021-03-23 10:17:33  INFO] checkpoints: [None, None, None, None, None]
[2021-03-23 10:17:33  INFO] padding: 20
[2021-03-23 10:17:33  INFO] add_joint_loss: False
[2021-03-23 10:17:33  INFO] debug: False
[2021-03-23 10:17:33  INFO] step1_epoch: 5
[2021-03-23 10:17:33  INFO] step2_epoch: 10
[2021-03-23 10:17:33  INFO] step3_epoch: 100
[2021-03-23 10:17:33  INFO] input_width: 576
[2021-03-23 10:17:33  INFO] input_height: 768
[2021-03-23 10:17:33  INFO] uid: HRNet_W48
[2021-03-23 10:17:33  INFO] seed: 20210309
[2021-03-23 10:17:33  INFO] inference: True


---

## 데이터셋 준비

In [5]:
dl_train, dl_valid, dl_test = get_pose_datasets(config, 1)

---

## 모델, 데이터셋 생성

In [6]:
import importlib

In [7]:
main = importlib.import_module("main-hrnet-train")

In [8]:
trainer = main.PoseTrainer(config, 1, "results/hrnet-train/ckpt-HRNet_W48_1.pth")

Load pretrained results/hrnet-train/ckpt-HRNet_W48_1.pth


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.70 GiB total capacity; 1.16 GiB already allocated; 15.81 MiB free; 1.23 GiB reserved in total by PyTorch)

---

## 학습 시작

In [16]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
indices = list(kf.split(total_imgs))

# train dataset
train_idx, valid_idx = indices[0]
ds_train = KeypointDataset(
    total_imgs[train_idx],
    total_keypoints[train_idx],
    augmentation=True,
    padding=PADDING,
)
dl_train = DataLoader(ds_train, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)

# validation dataset
ds_valid = KeypointDataset(
    total_imgs[valid_idx],
    total_keypoints[valid_idx],
    augmentation=False,
    padding=PADDING,
)
dl_valid = DataLoader(ds_valid, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

# test dataset
ds_test = TestKeypointDataset(test_imgs, offsets, ratios, augmentation=False)
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

epoch = START_EPOCH

In [17]:
if CHECKPOINT_PATH is not None:
    load(CHECKPOINT_PATH)

In [18]:
utils.seed_everything(SEED, deterministic=False)
RESULT_DIR.mkdir(parents=True, exist_ok=True)
log = utils.CustomLogger(RESULT_DIR / f"log_{UID}.log", "a")
log.info("학습 시작")
log.info("POSE_MODEL:", POSE_MODEL)
log.info("UID:", UID)
log.info("SEED:", SEED)
log.info("LR:", LR)
log.info("BATCH_SIZE:", BATCH_SIZE)
log.info("START_EPOCH:", START_EPOCH)
log.info("SAM:", SAM)
log.info("FOLD:", FOLD)
log.info("PADDING:", PADDING)
log.info("ADD_JOINT_LOSS:", ADD_JOINT_LOSS)
log.info("DEBUG:", DEBUG)
log.info("STEP1_EPOCHS:", STEP1_EPOCHS)
log.info("STEP2_EPOCHS:", STEP2_EPOCHS)
log.info("STEP3_EPOCHS:", STEP3_EPOCHS)
log.info("INPUT_WIDTH:", INPUT_WIDTH)
log.info("INPUT_HEIGHT:", INPUT_HEIGHT)
log.info("CHECKPOINT_PATH:", CHECKPOINT_PATH)
log.flush()

[2021-03-21 23:53:17  INFO] 학습 시작
[2021-03-21 23:53:17  INFO] POSE_MODEL: HRNet-W48
[2021-03-21 23:53:17  INFO] UID: 20210321-235315
[2021-03-21 23:53:17  INFO] SEED: 20210309
[2021-03-21 23:53:17  INFO] LR: 0.0001
[2021-03-21 23:53:17  INFO] BATCH_SIZE: 10
[2021-03-21 23:53:17  INFO] START_EPOCH: 1
[2021-03-21 23:53:17  INFO] SAM: False
[2021-03-21 23:53:17  INFO] FOLD: 3
[2021-03-21 23:53:17  INFO] PADDING: 30
[2021-03-21 23:53:17  INFO] ADD_JOINT_LOSS: False
[2021-03-21 23:53:17  INFO] DEBUG: False
[2021-03-21 23:53:17  INFO] STEP1_EPOCHS: 10
[2021-03-21 23:53:17  INFO] STEP2_EPOCHS: 20
[2021-03-21 23:53:17  INFO] STEP3_EPOCHS: 500
[2021-03-21 23:53:17  INFO] INPUT_WIDTH: 576
[2021-03-21 23:53:17  INFO] INPUT_HEIGHT: 768
[2021-03-21 23:53:17  INFO] CHECKPOINT_PATH: None


In [19]:
def train_loop():
    pose_model.train()

    meanloss, meanrmse = utils.AverageMeter(), utils.AverageMeter()
    with tqdm(total=len(dl_train.dataset), ncols=100, leave=False, file=sys.stdout, desc=f"Train {epoch:03d}") as t:
        for files, imgs, keypoints, target_heatmaps, ratios in dl_train:
            imgs_, target_heatmaps_ = imgs.cuda(), target_heatmaps.cuda()
            pred_heatmaps_ = pose_model(imgs_)
            loss = criterion(pred_heatmaps_, target_heatmaps_)
            rmse = criterion_rmse(pred_heatmaps_, target_heatmaps_, ratios.cuda())

            optimizer.zero_grad()
            loss.backward()
            if isinstance(optimizer, utils.SAM):
                optimizer.first_step()
                criterion(pose_model(imgs_), target_heatmaps_).backward()
                optimizer.second_step()
            else:
                optimizer.step()

            meanloss.update(loss.item())
            meanrmse.update(rmse.item())
            t.set_postfix_str(f"loss: {loss.item():.6f}, rmse: {rmse.item():.6f}", refresh=False)
            t.update(len(imgs))

    return meanloss(), meanrmse()

In [20]:
@torch.no_grad()
def valid_loop():
    pose_model.eval()

    meanloss, meanrmse = utils.AverageMeter(), utils.AverageMeter()
    with tqdm(total=len(dl_valid.dataset), ncols=100, leave=False, file=sys.stdout, desc=f"Valid {epoch:03d}") as t:
        for files, imgs, keypoints, target_heatmaps, ratios in dl_valid:
            imgs_, target_heatmaps_ = imgs.cuda(), target_heatmaps.cuda()
            pred_heatmaps_ = pose_model(imgs_)
            loss = criterion(pred_heatmaps_, target_heatmaps_)
            rmse = criterion_rmse(pred_heatmaps_, target_heatmaps_, ratios.cuda())

            meanloss.update(loss.item())
            meanrmse.update(rmse.item())
            t.set_postfix_str(f"loss: {loss.item():.6f}, rmse: {rmse.item():.6f}", refresh=False)
            t.update(len(imgs))

    return meanloss(), meanrmse()

### Finetune Step 1

In [21]:
log.info("Finetune Step 1")
torch.cuda.empty_cache()
pose_model.freeze_head()
for epoch in range(epoch, STEP1_EPOCHS + 1):
    tloss, trmse = train_loop()
    vloss, vrmse = valid_loop()

    log.info(f"Epoch: {epoch:03d}, loss: {tloss:.6f};{vloss:.6f}, rmse {trmse:.6f};{vrmse:.6f}")
    log.flush()

    if best_loss > vloss:
        best_loss = vloss
        save(RESULT_DIR / f"ckpt-{UID}_{FOLD}.pth")

[2021-03-21 23:53:18  INFO] Finetune Step 1
[2021-03-21 23:58:08  INFO] Epoch: 001, loss: 7.564656;5.685315, rmse 101.205292;79.330643 
[2021-03-22 00:02:55  INFO] Epoch: 002, loss: 4.890828;4.309723, rmse 58.377068;34.204077  
[2021-03-22 00:07:43  INFO] Epoch: 003, loss: 3.989645;3.856502, rmse 30.450904;20.512082  
[2021-03-22 00:12:31  INFO] Epoch: 004, loss: 3.567925;3.667795, rmse 20.836941;18.546650  
[2021-03-22 00:17:19  INFO] Epoch: 005, loss: 3.271069;3.543567, rmse 17.804651;16.872146  
[2021-03-22 00:22:07  INFO] Epoch: 006, loss: 3.025566;3.469329, rmse 14.656848;16.329735  
[2021-03-22 00:26:55  INFO] Epoch: 007, loss: 2.816415;3.416607, rmse 14.205641;14.817151  
[2021-03-22 00:31:43  INFO] Epoch: 008, loss: 2.615595;3.427628, rmse 12.346229;15.498619  
[2021-03-22 00:36:31  INFO] Epoch: 009, loss: 2.437437;3.429644, rmse 11.747901;14.390473  
[2021-03-22 00:41:18  INFO] Epoch: 010, loss: 2.280654;3.440171, rmse 11.449862;14.956587  


### Finetune Step 2

In [23]:
log.info("Finetune Step 2")
torch.cuda.empty_cache()
pose_model.freeze_tail()
for epoch in range(epoch, STEP2_EPOCHS + 1):
    tloss, trmse = train_loop()
    vloss, vrmse = valid_loop()

    log.info(f"Epoch: {epoch:03d}, loss: {tloss:.6f};{vloss:.6f}, rmse {trmse:.6f};{vrmse:.6f}")
    log.flush()

    if best_loss > vloss:
        best_loss = vloss
        save(RESULT_DIR / f"ckpt-{UID}_{FOLD}.pth")

[2021-03-22 08:53:04  INFO] Finetune Step 2
[2021-03-22 08:55:56  INFO] Epoch: 010, loss: 2.104729;3.616645, rmse 10.008939;14.551863  
[2021-03-22 08:58:47  INFO] Epoch: 011, loss: 2.084577;3.634394, rmse 9.649983;14.678430   
[2021-03-22 09:01:37  INFO] Epoch: 012, loss: 2.076373;3.658820, rmse 9.464002;14.239340   
[2021-03-22 09:04:29  INFO] Epoch: 013, loss: 2.081457;3.664402, rmse 9.375424;14.112958   
[2021-03-22 09:07:17  INFO] Epoch: 014, loss: 2.078772;3.683389, rmse 8.685960;13.593442   
[2021-03-22 09:10:08  INFO] Epoch: 015, loss: 2.077808;3.697917, rmse 8.327278;14.109696   
[2021-03-22 09:12:58  INFO] Epoch: 016, loss: 2.071069;3.697929, rmse 8.271958;14.043605   
[2021-03-22 09:15:48  INFO] Epoch: 017, loss: 2.071886;3.706438, rmse 8.377780;13.921576   
[2021-03-22 09:18:44  INFO] Epoch: 018, loss: 2.081219;3.690305, rmse 8.040496;13.749948   
[2021-03-22 09:21:36  INFO] Epoch: 019, loss: 2.066945;3.700463, rmse 8.403259;13.889715   
[2021-03-22 09:24:29  INFO] Epoch: 0

### Finetune Step 3

In [ ]:
log.info("Finetune Step 3")
torch.cuda.empty_cache()
pose_model.unfreeze_all()
for epoch in range(epoch, STEP3_EPOCHS + 1):
    tloss, trmse = train_loop()
    vloss, vrmse = valid_loop()

    log.info(f"Epoch: {epoch:03d}, loss: {tloss:.6f};{vloss:.6f}, rmse {trmse:.6f};{vrmse:.6f}")
    log.flush()
    scheduler.step(vo.loss)

    if best_loss > vo.loss:
        best_loss = vo.loss
        earlystop_cnt = 0
        save(RESULT_DIR / f"ckpt-{UID}_{FOLD}.pth")
    elif earlystop_cnt >= 10:
        log.info(f"Stop training at epoch", epoch)
        break
    else:
        earlystop_cnt += 1

---

## 검증

In [ ]:
# 어떻게 학습됐는지 검증한번 해보자

with torch.no_grad():
    pose_model.eval()
    valid_output_dir = Path(f"results/hrnet_example/valid-{UID}")
    valid_output_dir.mkdir(parents=True, exist_ok=True)

    meanloss, meanrmse = utils.AverageMeter(), utils.AverageMeter()
    with tqdm(total=len(dl_valid.dataset), ncols=100, leave=True, file=sys.stdout, desc=f"Valid {epoch:03d}") as t:
        for files, imgs, keypoints, target_heatmaps, ratios in dl_valid:
            imgs_, target_heatmaps_ = imgs.cuda(), target_heatmaps.cuda()
            pred_heatmaps_ = pose_model(imgs_)
            loss = criterion(pred_heatmaps_, target_heatmaps_)
            rmse = criterion_rmse(pred_heatmaps_, target_heatmaps_, ratios.cuda())

            meanloss.update(loss.item())
            meanrmse.update(rmse.item())
            t.set_postfix_str(f"loss: {loss.item():.6f}, rmse: {rmse.item():.6f}", refresh=False)

            for file, img, pred_heatmap, target_heatmap, ratio in zip(
                files, imgs, pred_heatmaps_.cpu(), target_heatmaps, ratios
            ):
                file = Path(file)

                pred_keypoint = utils.heatmaps2keypoints(pred_heatmap)
                target_keypoint = utils.heatmaps2keypoints(target_heatmap)
                pred_keypoint = pred_keypoint * 4 / ratio.view(1, 2)
                target_keypoint = target_keypoint * 4 / ratio.view(1, 2)
                pred_keypoint = pred_keypoint.type(torch.int64).numpy()
                target_keypoint = target_keypoint.type(torch.int64).numpy()

                img_np = utils.denormalize(img).mul(255).type(torch.uint8).permute(1, 2, 0).numpy()
                ori_width = img.size(2) / ratio[0]
                ori_height = img.size(1) / ratio[1]
                img_np = cv2.resize(img_np, (ori_width, ori_height))

                pred_keypoint_img = utils.draw_keypoints(img_np, pred_keypoint)
                target_keypoint_img = utils.draw_keypoints(img_np, target_keypoint)

                rmse = np.sqrt(np.mean(np.square(pred_keypoint - target_keypoint)))

                imageio.imwrite(valid_output_dir / f"{file.name[:-4]}_pred_{rmse:.4f}.jpg", pred_keypoint_img)
                imageio.imwrite(valid_output_dir / f"{file.name[:-4]}_target.jpg", target_keypoint_img)

                t.update()